In [41]:
import pandas as pd
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
#from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

In [44]:
def neural_net(x_train_filename, x_test_filename, y_train_filename, y_test_filename):
    df_X_train = pd.read_csv(x_train_filename, header=None)
    df_Y_train = pd.read_csv(y_train_filename, header=None)

    df_X_test = pd.read_csv(x_test_filename, header=None)
    df_Y_test = pd.read_csv(y_test_filename, header=None)

    x_train = np.array(df_X_train.values)
    y_train = np.array(df_Y_train.values)

    x_test = np.array(df_X_test.values)
    y_test = np.array(df_Y_test.values)
    
    def get_model():
        nclass = 1
        inp = Input(shape=(x_train.shape[1],))

        dense_1 = Dense(50, activation = activations.relu  , name="dense_1")(inp)
        #dense_1 = Dense(100, activation = activations.relu  , name="dense_2")(dense_1)
        dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3")(dense_1)

        model = models.Model(inputs = inp , outputs = dense_1)
        opt = optimizers.Adam(0.001)

        model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
        model.summary()
        return model
    
    model = get_model()
    
    file_path = "neural_net.h5"
    checkpoint = ModelCheckpoint(file_path , monitor = 'val_acc', verbose=1, save_best_only=True, mode = 'max')
    early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
    redonplat = ReduceLROnPlateau(monitor="val_acc", mode = "max", patience = 3, verbose =2)
    callbacks_list = [checkpoint, early, redonplat]
    
    #Fitting part
    model.fit(x_train, y_train, epochs = 1000 , verbose=2, callbacks=callbacks_list, validation_split=0.1)
    model.load_weights(file_path)

    #Testing part 
    pred_test = model.predict(x_test)
    pred_test = (pred_test>0.5).astype(np.int8)

    f1 = f1_score(y_test, pred_test)

    print("Test f1 score : %s "% f1)

    #acc = accuracy_score(y_test, pred_test)

    #print("Test accuracy score : %s "% acc)

    roc = roc_auc_score(y_test, pred_test)

    print("Test roc score : %s "% roc)

    #auprc = average_precision_score(y_test, pred_test)

    #print("Test auprc score : %s "% auprc)

This is an example of how to use the function neural_net(x_train_filename, x_test_filename, y_train_filename, y_test_filename)

In [45]:
#neural_net("x_train.csv", "x_test.csv", "y_train.csv", "y_test.csv")

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 185)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                9300      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 9,351
Trainable params: 9,351
Non-trainable params: 0
_________________________________________________________________
Train on 5400 samples, validate on 600 samples
Epoch 1/1000
 - 1s - loss: 9.6422 - acc: 0.3952 - val_loss: 9.4326 - val_acc: 0.4083

Epoch 00001: val_acc improved from -inf to 0.40833, saving model to neural_net.h5
Epoch 2/1000
 - 0s - loss: 9.6422 - acc: 0.3952 - val_loss: 9.4326 - val_acc: 0.4083

Epoch 00002: val_acc did not improve from 0.40833
Epoch 3/1000
 - 0s - loss: 9.6422 -